# USA Health Outcomes study

<span style="color:red"> Svetlana Maslenkova </span>

## Goal of the project
For this case study, you will perform a series of analyses to clarify the factors associated with population health outcomes in the United States. More specifically, the goal of your analysis is to understand how the health outcomes of US counties are associated with the county's characteristics: demographic, geographic, economic, behavioral, etc.   

## Data
All analysis for this case study will use the [PLACES Dataset](https://chronicdata.cdc.gov/500-Cities-Places/PLACES-County-Data-GIS-Friendly-Format-2022-releas/i46a-9kgh). The PLACES dataset contains information on the prevalence of health conditions across US counties. Some of the database columns contain information about general health (e.g. `GHLTH_CrudePrev` describe the prevalence of general health issues) while other columns refer to specific health conditions (e.g. `DEPRESSION_CrudePrev` describes the prevalence of depression). In addition to that, other datasets supplement the data from PLACES with additional information to help in answering the questions herein (e.g. data from the US Census, CDC, etc.).

<hr>

## 1. Data Task 

Study the data described in `Section 0.4`; then select one or more publically available datasets that provide characteristics of US counties that you think will be associated with the health outcomes reported in the PLACES Dataset; be prepared to justify your dataset selection. Merge your selected datasets with the data from the PLACES Dataset into a single DataFrame.

### Cells for working on Google Colab

In [ ]:
# # # for google colab
# from google.colab import drive
# drive.mount('/content/drive')

# # define the path to data
# data_path = '/content/drive/MyDrive/Colab_Notebooks/Ghamut_assessment/data/'

### Importing data

In [ ]:
import os

data_path = os.getcwd() + '/data/'

In [ ]:
import sys

sys.version

In [ ]:
# import packages
import pandas as pd
pd.options.mode.chained_assignment = None 

# loading PLACES data by county
places_data = pd.read_csv(data_path+'PLACES__County_Data__GIS_Friendly_Format___2022_release.csv')
places_data.columns = places_data.columns.str.lower()

# loading demographics data by county
demographics_data = pd.read_csv(data_path+'cc-est2021-all.csv', encoding='latin-1')
demographics_data = demographics_data[demographics_data.YEAR==1]
demographics_data.columns = demographics_data.columns.str.lower()
# get sample for a preview
sample_demo_data = demographics_data.iloc[10, :]

# loading poverty data by county
poverty_data = pd.read_excel(data_path+'PovertyEstimates.xlsx', header=4) 
poverty_data.columns = poverty_data.columns.str.lower()

# loading unemployment data by county
unemployment_data = pd.read_excel(data_path+'Unemployment.xlsx', header=4, usecols="A:F,CJ:CR") 
unemployment_data.columns = unemployment_data.columns.str.lower()

# loading education data by county
education_data = pd.read_excel(data_path+'Education.xlsx', header=3, usecols="A:C,F,G,AV:BC") 
education_data.columns = education_data.columns.str.lower()

# loading social vulnerability data by county
svi_data = pd.read_csv(data_path+'Social_Vulnerability_Index_2018_-__United_States__county.csv')
svi_data.columns = svi_data.columns.str.lower()

# loading mortality data by county
with  open(data_path+'Multiple Cause of Death, 2018-2021, Single Race.txt', 'r') as f:
    lines = [line.replace('\n', '').split('\t') for line in f.readlines()]
mort_data = pd.DataFrame(lines[1:], columns=lines[0])
mort_data.columns = mort_data.columns.str.lower().str.replace('"', '')
mort_data = mort_data.iloc[:3114, :]

# loading sunshine data by state
sunshine_data = pd.read_csv(data_path+'sunshine_data.csv')
sunshine_data.columns = sunshine_data.columns.str.lower()

### Preprocessing

In [ ]:
import numpy as np

# remove some columns from PLACES and keep only crude prevalence ones
data_pl = places_data.loc[:, ~(places_data.columns.str.contains('95ci|adjprev'))]

## demographics data
demographics_columns = ['STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'AGEGRP', 'TOT_POP', 'TOT_MALE', 'TOT_FEMALE', 'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE', 'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE'] 
# sum female and male populations by race
WA = demographics_data.wa_male + demographics_data.wa_female
BA = demographics_data.ba_male + demographics_data.ba_female
IA = demographics_data.ia_male + demographics_data.ia_female
AA = demographics_data.aa_male + demographics_data.aa_female
NA = demographics_data.na_male + demographics_data.na_female
TOM = demographics_data.tom_male + demographics_data.tom_female
# get new dataframe for demographics with only necessary columns
demo_data = demographics_data[['state', 'county', 'stname', 'ctyname', 'agegrp', 'tot_pop', 'tot_male', 'tot_female']]
demo_data.loc[:, 'race_wa'] = WA
demo_data.loc[:, 'race_ba'] = BA
demo_data.loc[:, 'race_ia'] = IA
demo_data.loc[:, 'race_aa'] = AA
demo_data.loc[:, 'race_na'] = NA
demo_data.loc[:, 'race_tom'] = TOM
# rename gender columns 
demo_data = demo_data.rename(columns={'tot_male':'male', 'tot_female':'female', 'tot_pop':'population'})
# map age groups to a smaller set
demo_data.agegrp = demo_data.agegrp.apply(lambda x: 'agegrp_1' if x in range(1,5) else 'agegrp_2' if x in range(5,9) else 'agegrp_3' if x in range(9,14) else 'agegrp_4' if x in range(14,19) else 'agegrp_tot')
# get population by race, gender and age group (for EDA)
population_by_race_gender_agegroup = pd.pivot_table(demo_data, index=['state', 'county'], columns=['agegrp'], aggfunc='sum')
population_by_race_gender_agegroup.columns = ['{}_{}'.format(t, v) for v,t in population_by_race_gender_agegroup.columns]
population_by_race_gender_agegroup = population_by_race_gender_agegroup.reset_index()
# get population by age groups 
population_by_age = demo_data[~(demo_data.agegrp=='agegrp_tot')].groupby(['state', 'county', 'agegrp']).agg('sum')['population'].reset_index()
population_by_age = pd.pivot_table(population_by_age, index=['state', 'county'], columns='agegrp')
population_by_age.columns = ['{}_{}'.format(t, v) for v,t in population_by_age.columns]
population_by_age = population_by_age.reset_index()
# keep only total population rows for race columns
demo_data = demo_data[demo_data.agegrp=='agegrp_tot'].drop(columns='agegrp')
# concatenate with age groups columns
demo_data = demo_data.merge(population_by_age, how='inner', on=['state', 'county'])
# make fips codes standard
demo_data.county = demo_data.county.astype(str).apply(lambda x: '00'+ x if len(x)==1 else '0'+x if len(x)==2 else x)
demo_data['countyfips'] = demo_data[['state', 'county']].apply(lambda row: str(row.state)+row.county, axis=1).astype('int64')
demo_data.insert(3, 'countyfips', demo_data.pop('countyfips'))
# calculate percent of population of a particular race as  % of  total population in a county
race_percent_dict = {}
for r in ['race_ba',	'race_ia',	'race_aa',	'race_na',	'race_tom', 'race_wa']:
  name = r + '_%'
  race_percent_dict[name] = demo_data.apply(lambda row: np.round(row[r]/row['population'], 2)*100, axis=1).to_list()
race_data_percent_df = pd.DataFrame(race_percent_dict)
# calculate percent of population of a particular race as  % of  total population of this race in a county (to see the life length of representatives of each race)
race_by_age_percent_dict = {}
for r in ['aa', 'ba', 'ia', 'na', 'wa', 'tom']:
  for a in ['1', '2', '3', '4']:
    name = 'race_' + r + '_agegrp_' + a + '%'
    race_by_age_percent_dict[name] = population_by_race_gender_agegroup.apply(lambda row: np.round(row['agegrp_'+a+'_race_'+r]/(row['agegrp_tot_race_'+r]+0.00001), 2)*100, axis=1).to_list()
race_by_age_percent_df = pd.DataFrame(race_by_age_percent_dict)
# combine with demographics dataframe
demo_data = pd.concat([demo_data, race_data_percent_df, race_by_age_percent_df], axis=1)

# Unemploymnet data
unemp_columns = ['fips_code', 'unemployment_rate_2020', 'median_household_income_2020']
unemp_data = unemployment_data[unemp_columns].rename(columns={'fips_code':'countyfips'})

# mortality data
mrt_data = mort_data.rename(columns={'county code':'countyfips', 'crude rate':'mort_cruderate'})
mrt_data.countyfips = mrt_data.countyfips.apply(lambda x: x.replace('"', '')).astype('int64')
# mrt_data = mrt_data[['countyfips', 'deaths', 'mort_cruderate']]
mrt_data.deaths = mrt_data.deaths.astype('int64')
mort_data.state = mort_data.state.fillna('').apply(lambda x: x.replace('"', ''))
# convert deaths to crude rates per 1000 
mrt_data = mort_data.rename(columns={'county code':'countyfips', 'crude rate':'mort_cruderate'})
mrt_data.countyfips = mrt_data.countyfips.apply(lambda x: x.replace('"', '')).astype('int64')
mrt_data['mort_cruderate'] = mrt_data.apply(lambda row: (int(row['deaths'])/int(row['population']))*1000, axis=1)

# poverty data
pov_data = poverty_data.rename(columns={'fips_code':'countyfips'})[['stabr', 'countyfips','area_name', 'povall_2020', 'pctpovall_2020']]
pov_data = pov_data[pov_data.countyfips.isin(data_pl.countyfips.values)]

# education data
# Federal Information Processing Standard (FIPS) Code, 2013 Rural-urban Continuum Code, 2013 Urban Influence Code, Percent of adults with less than a high school diploma, 2017-21, Percent of adults with a high school diploma only, 2017-21, Percent of adults completing some college or associate's degree, 2017-21
edu_data = education_data.rename(columns={'Federal Information Processing Standard (FIPS) Code'.lower():'countyfips', \
                                          '2013 Rural-urban Continuum Code'.lower():'rural_urban_cont_code',\
                                           '2013 Urban Influence Code'.lower():'urban_inf_code',   \
                                            'Percent of adults with less than a high school diploma, 2017-21'.lower():'adults_no_hs_diploma_percent',\
                                            'Percent of adults with a high school diploma only, 2017-21'.lower():'adults_hs_diploma_only_percent',\
                                              "Percent of adults completing some college or associate's degree, 2017-21".lower():'adults_college_diploma_percent',\
                                                "Percent of adults with a bachelor's degree or higher, 2017-21".lower():'adults_bachelors_or_higher_percent'})
edu_data = edu_data[['countyfips', 'rural_urban_cont_code', 'urban_inf_code', \
                     'adults_no_hs_diploma_percent', 'adults_hs_diploma_only_percent', \
                        'adults_college_diploma_percent', 'adults_bachelors_or_higher_percent' ]] 

# SVI data
# FIPS,EP_DISABL,EP_SNGPNT,EP_MINRTY,EP_LIMENG,EP_NOVEH,EP_POV,F_DISABL,F_SNGPNT,F_MINRTY,F_LIMENG,F_NOVEH,F_POV,
svi_columns = "FIPS,EP_DISABL,EP_SNGPNT,EP_MINRTY,EP_LIMENG,EP_NOVEH,EP_POV,F_DISABL,F_SNGPNT,F_MINRTY,F_LIMENG,F_NOVEH,F_POV".lower().split(',')
sv_data = svi_data[svi_columns].rename(columns={'fips':'countyfips'})
sv_data.loc[:, sv_data.columns.str.contains('^f', regex=True)].clip(0,1, inplace=True)
sv_data.clip(0, inplace=True)

# sunshine data
sunshine_data['% sun'] = sunshine_data['% sun'].apply(lambda x: x.replace('–', '-1')).astype(int)
sunshine_data['total hours'] = sunshine_data['total hours'].apply(lambda x: x.replace('–', '-1')).astype(int)

### Merge datasets together

In [ ]:
# merge places with demographics dataset
data = pd.merge(data_pl, demo_data.drop(columns=['state', 'county', 'stname', 'ctyname']), how='left', on='countyfips')
# merge unemployment with places data
data = pd.merge(data, unemp_data, how='left', on='countyfips')
# merge mortality with places data
data = pd.merge(data, mrt_data, how='left', on='countyfips')
# merge poverty with places data
data = pd.merge(data, pov_data, how='left', on='countyfips')
# merge education data with places data
data = pd.merge(data, edu_data, how='left', on='countyfips')
# merge SVI data  with places data
data = pd.merge(data, sv_data, how='left', on='countyfips')
# merge with sunshine data
data = data.merge(sunshine_data.rename(columns={'state':'statedesc'}), how='left', on='statedesc')

data['countyfips'] = data['countyfips'].apply(lambda x: '0'+str(x) if len(str(x))<5 else x)

print('The shape of the resulting dataframe is: ', data.shape)

## 2. Visualization Task
Using the data you created in Section `1.0`, perform an Exploratory Data Analysis (EDA). EDA consists of generating several cross tabulations, summary statistics and graphical representations of the data that help:

1. Identify patterns in county characteristics that are associated with the prevalence of health conditions.
2. Detect oddities/errors in the data (if any) that may confound the results of a model trained to predict health conditions (if not accounted for).

### Health conditions

In [ ]:
# group columns by their meanings
risk_behaviors = ['binge_crudeprev', 'csmoking_crudeprev', 'sleep_crudeprev', 'lpa_crudeprev']
prevention = ['access2_crudeprev', 'checkup_crudeprev', 'dental_crudeprev', 'bpmed_crudeprev', 'cholscreen_crudeprev', 'mammouse_crudeprev', 'cervical_crudeprev',\
              'corem_crudeprev', 'corew_crudeprev', 'colon_screen_crudeprev']
outcomes = ['arthritis_crudeprev', 'casthma_crudeprev', 'bphigh_crudeprev', 'cancer_crudeprev', 'highchol_crudeprev', 'kidney_crudeprev', 'copd_crudeprev',\
            'chd_crudeprev', 'diabetes_crudeprev', 'depression_crudeprev', 'obesity_crudeprev', 'stroke_crudeprev', 'teethlost_crudeprev']
health_status = ['ghlth_crudeprev', 'phlth_crudeprev', 'mhlth_crudeprev']

Leading health risk behaviors among USA counties are **sleeping less than 7 hours and lack of physical activity**. The prevalent health outcomes are **high blood pressure and cholesterol, obesity**, where obesity have the highest maximum crude rate value of around 52 ⬇️ 

In [ ]:
print('Shape of places dataset is: ', data_pl.shape)
data_pl[risk_behaviors].describe()

In [ ]:
data_pl[health_status].describe()

In [ ]:
data_pl[outcomes].describe()

#### Obesity

Obesity is one of the most prevalent health outcomes across US counties. This ailment is more common in the south-eastern states of US and Alaska ⬇️

In [ ]:
data_pl.nlargest(5, 'obesity_crudeprev')[['statedesc', 'countyname', 'obesity_crudeprev']]

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
import pandas as pd
import plotly.express as px

fig = px.choropleth_mapbox(data, geojson=counties, locations='countyfips', color='obesity_crudeprev',
                           color_continuous_scale="Viridis",
                           range_color=(17, 40),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'obesity_crudeprev':'obesity rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

⬆️ Obesity crude prevalence rates

#### Health insurance access

Rates of people **lacking health insurance** are higher in **southern parts of the country** and in some areas in **Alaska**. There are high rates for **cholesterol screening** and **taking blood pressure medications**, which is consistent with the health outcomes data. In addition, we can see high rates for **cervical cancer screening** and **mammography use** among **women**. However, the rates for adult men and women who are up to date on **core set of clinical preventive services** are quite low, comparing with other preventive measures rates⬇️

In [ ]:
data_pl[prevention].describe()

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
import pandas as pd
import plotly.express as px

fig = px.choropleth_mapbox(data, geojson=counties, locations='countyfips', color='access2_crudeprev',
                           color_continuous_scale="Viridis",
                           range_color=(5, 25),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'access2_crudeprev':'current lack of health insurance'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

⬆️ Lack of health insurance crude prevalence rates

#### Health and health risk behaviors

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

# convert values of health risk behaviors rates to categorical variables, 
# where 'low' category corresponds to values below 1st quartile, moderate between 1st and 3rd, and high - above 3rd quartile
# data_pl[['lpa_crudeprev', 'sleep_crudeprev']].describe()
lpa_cat = data_pl['lpa_crudeprev'].apply(lambda x: 'high' if x<24 else 'moderate' if x<=30 else 'low')
sleep_cat = data_pl['sleep_crudeprev'].apply(lambda x: 'high' if x<30 else 'moderate' if x<=36 else 'low')
# data_pl[['binge_crudeprev', 'csmoking_crudeprev']].describe()
binge_cat = data_pl['binge_crudeprev'].apply(lambda x: 'low' if x<10 else 'moderate' if x<=18 else 'high')
smoking_cat = data_pl['csmoking_crudeprev'].apply(lambda x: 'low' if x<10 else 'moderate' if x<=18 else 'high')

# plot cross tabulations as heatmaps
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(10,8))
title = fig.suptitle('Еhe effect of health risk behaviors on health rates\n', fontsize=16)

cross_tab_sleep_lpa_phlth = sns.heatmap(pd.crosstab(lpa_cat, sleep_cat, data_pl['phlth_crudeprev'], \
                                                    aggfunc='mean'), annot=True, cmap="Blues", ax=ax1)
_ = cross_tab_sleep_lpa_phlth.set_xlabel("physical activity",fontsize=12)
_ = cross_tab_sleep_lpa_phlth.set_ylabel("sleep quality", fontsize=12)
_ = cross_tab_sleep_lpa_phlth.set_title("Poor physical health rate", fontsize=15)

cross_tab_sleep_lpa_mhlth = sns.heatmap(pd.crosstab(lpa_cat, sleep_cat, data_pl['mhlth_crudeprev'], \
                                                    aggfunc='mean'), annot=True, cmap="Blues", ax=ax2)
_ = cross_tab_sleep_lpa_mhlth.set_xlabel("physical activity",fontsize=12)
_ = cross_tab_sleep_lpa_mhlth.set_ylabel("sleep quality", fontsize=12)
_ = cross_tab_sleep_lpa_mhlth.set_title("Poor mental health rate", fontsize=15)

cross_tab_binge_smoking_phlth = sns.heatmap(pd.crosstab(binge_cat, smoking_cat, data_pl['phlth_crudeprev'], \
                                                    aggfunc='mean'), annot=True, cmap="Blues", ax=ax3)
_ = cross_tab_binge_smoking_phlth.set_xlabel("binge drinking rate",fontsize=12)
_ = cross_tab_binge_smoking_phlth.set_ylabel("currently smoking rate", fontsize=12)
_ = cross_tab_binge_smoking_phlth.set_title("Poor physical health rate", fontsize=15)

cross_tab_binge_smoking_mhlth = sns.heatmap(pd.crosstab(binge_cat, smoking_cat, data_pl['mhlth_crudeprev'], \
                                                    aggfunc='mean'), annot=True, cmap="Blues", ax=ax4)
_ = cross_tab_binge_smoking_mhlth.set_xlabel("binge drinking rate",fontsize=12)
_ = cross_tab_binge_smoking_mhlth.set_ylabel("currently smoking rate", fontsize=12)
_ = cross_tab_binge_smoking_mhlth.set_title("Poor mental health rate", fontsize=15)

fig.tight_layout()

⬆️ The heatmaps above shows the mean rates of self-reported physical and mental health, depending on the health risk behaviors rates.


We can see that lack of sleep and physical activity do affect all types of people's health. However, low and moderate **sleep quality** correlates with **physical health** the most, whereas **physical inactivity** has strong association with **mental health**.


High **binge drinking** rates correlates with all types of health, especially with **mental health** of the population.


💡 We also see that **smoking** rates **positively correlate with health rates**, which is not obvious. The reason might be behind the survey question itself. The description of the given rate is formulated as follows:  "*Respondents aged ≥18 years who report having smoked ≥100 cigarettes in their lifetime and currently smoke every day or some days.*"
Therefore, people who had been smoking for a long time but currently do not smoke (probably because of medical reasons) - also considered as not smoking in this survey, which can inject some **noise** in the population smoking rates.

### Mortality

In [ ]:
print('Shape of mortality dataset is: ', mort_data.shape)
mrt_data[mrt_data['mort_cruderate'] != -1].describe()

#### States with highest mortality rates

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

# plt.ioff()
sns.set(context="paper")

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(14,6), sharey=True)

mort_barplot = sns.barplot(data=data[data['mort_cruderate'] != -1][['statedesc', 'mort_cruderate']].groupby('statedesc')\
                           .mean().reset_index().sort_values('mort_cruderate', ascending=False).head(15), \
                           x='statedesc', y='mort_cruderate', palette='muted', ax=ax1)
ticks = plt.setp(mort_barplot.get_xticklabels(), rotation=90, size=12)
mort_barplot.set_ylabel('Mortality rates by state', fontsize=12)
mort_barplot.set_xlabel('', fontsize=12)
title = mort_barplot.set_title('States with the highest mortality rates\n', fontsize=20)

mort_barplot2 = sns.barplot(data=data[data['mort_cruderate'] != -1][['statedesc', 'mort_cruderate']].groupby('statedesc')\
                           .mean().reset_index().sort_values('mort_cruderate', ascending=True).head(15), \
                           x='statedesc', y='mort_cruderate', palette='muted', ax=ax2)
ticks = plt.setp(mort_barplot2.get_xticklabels(), rotation=90, size=12)
mort_barplot2.set_ylabel('Mortality rates by state', fontsize=12)
mort_barplot2.set_xlabel('', fontsize=12)
title = mort_barplot2.set_title('States with the lowest mortality rates\n', fontsize=20)

fig.tight_layout()

Majority of countries have mortality rates between 11 and 15 per 1000. *Mississippi, Arkansas, Alabama, West Virginia, Tennessee* - are states with the **highest mortality** rates in counties. On the other hand, *Alaska, Utah, Hawaii, District of Columbia, California, Washington* - have the **lowest mortality** rates in their counties.

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
import pandas as pd
import plotly.express as px

fig = px.choropleth_mapbox(data[data['mort_cruderate']!=-1], geojson=counties, locations='countyfips', color='mort_cruderate',
                           color_continuous_scale="Viridis",
                           range_color=(1, 20),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'mort_cruderate':'mortality rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

⬆️ Mortality rate

#### Mortality and Social Vulnerability

In [ ]:
data[['ep_minrty', 'ep_disabl', 'ep_sngpnt', 'ep_limeng', 'ep_noveh', 'ep_pov', 'mort_cruderate']].describe()

In [ ]:
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

# convert values of health risk behaviors rates to categorical variables, 
minority_cat = data['ep_minrty'].apply(lambda x: 'low' if x<5 else 'moderate' if x<=15 else 'high')
disabl_cat = data['ep_disabl'].apply(lambda x: 'low' if x<5 else 'moderate' if x<=15 else 'high')
sngpnt_cat = data['ep_sngpnt'].apply(lambda x: 'low' if x<5 else 'moderate' if x<=15 else 'high')
limeng_cat = data['ep_limeng'].apply(lambda x: 'low' if x<5 else 'moderate' if x<=15 else 'high')
noveh_cat = data['ep_noveh'].apply(lambda x: 'low' if x<5 else 'moderate' if x<=15 else 'high')
pov_cat = data['ep_pov'].apply(lambda x: 'low' if x<5 else 'moderate' if x<=15 else 'high')

# plot cross tabulations as heatmaps
fig, ((ax1, ax2, ax3)) = plt.subplots(nrows=1, ncols=3, figsize=(13,4))
title = fig.suptitle('Social vulnerability and mortality rates\n', fontsize=16)

cross_tab_mort_mnrt_pov = sns.heatmap(pd.crosstab(minority_cat, pov_cat, data['mort_cruderate'], \
                                                    aggfunc='mean'), annot=True, cmap="BuPu", ax=ax1)
_ = cross_tab_mort_mnrt_pov.set_xlabel("poverty level",fontsize=12)
_ = cross_tab_mort_mnrt_pov.set_ylabel("minority representatives rate", fontsize=12)


cross_tab_mort_disabl_noveh = sns.heatmap(pd.crosstab(disabl_cat, noveh_cat, data['mort_cruderate'], \
                                                    aggfunc='mean'), annot=True, cmap="BuPu", ax=ax2)
_ = cross_tab_mort_disabl_noveh.set_xlabel("households with no vehicle rate",fontsize=12)
_ = cross_tab_mort_disabl_noveh.set_ylabel("population with disabilities rate", fontsize=12)

cross_tab_mort_snglpnt_limeng = sns.heatmap(pd.crosstab(sngpnt_cat, limeng_cat, data['mort_cruderate'], \
                                                    aggfunc='mean'), annot=True, cmap="BuPu", ax=ax3)
_ = cross_tab_mort_snglpnt_limeng.set_xlabel("'english 'less than well'",fontsize=12)
_ = cross_tab_mort_snglpnt_limeng.set_ylabel("single parent households", fontsize=12)


fig.tight_layout()

Several features addressing social vulnerability of the population are linked to mortality rates. For example, high **poverty** levels and high population with **disabilities** rates are associated with **higher mortality**.


💡*We also can notice that counties with a low rate of people who speak English 'less than well' have higher mortality, and counties with more people speaking English well have higher mortality. This phenomenon can be related to the fact that people starting from 5 years old were counted in for this rate estimation. Children born in non-English speaking families most probably learn English much later in life. Therefore, a community with a higher number of such children has a higher 'English less than well' rate. However, the mean age is lower in that community, which may result in the lower mortality rate (since people pass away in older age normally).*

### Demographics

In [ ]:
print('Shape of demographics dataset is: ', demo_data.shape)
demo_data.describe()

#### Population by gender

In [ ]:
demo_data.nlargest(3, 'population')[['ctyname', 'population']]

Median population of male and female is around 13.000, with **male** median population is slightly **higher** across counties. Majority of counties have population between **11.000** and **70.000**. However, there are some outliers in terms of population sizes, such as large counties inside big cities such *Los Angeles County*, *Cook County*, *Harris County*, and others.

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(20.,5.)})
population_boxplot =  sns.boxplot(data=demo_data[['population', 'male', 'female']].clip(0, demo_data.population.describe()['75%']), orient="h")
xticks = population_boxplot.set_xticks(range(0, int(demo_data.population.describe()['75%']), 5000))
title = population_boxplot.set_title('Population distribution by gender across USA counties\n', fontsize=20)

#### Population by race

The people of white race make up between 78% and 95% of the population in the majority of counties. However, we see that in some counties the majority belong to other races (outlier dots on the boxplot), such as black or american indian and alaska native. Small part (<10%) of the population consists of people of two and more races.


Key to race coding map:
- ba - black of african alone
- ia - american indian and alaska native alone
- aa - asian alone
- na - native hawaiian and other pacific islander alone
- wa - white alone
- tom - two or more races

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(20.,5.)})
population_boxplot =  sns.boxplot(data=race_data_percent_df, orient="h")
xticks = population_boxplot.set_xticks(range(0, 110, 10))
title = population_boxplot.set_title('Population distribution by race across USA counties, % of total county population\n', fontsize=20)

#### Race and length of life 

In [ ]:
import matplotlib.pyplot as plt 

race_by_age_group_visual = demo_data.loc[:, demo_data.columns.str.contains(r'(^race.*\d%$)|(state)|(county)', regex=True)]

labels = ['0-19 years', '20-39 years', '40-64 years', '65+ years']
colors = ['#88d8b0', '#ffcc5c', '#ff6f69', '#4279a3']

fig, ((ax0, ax1, ax3), (ax4, ax5, ax6)) = plt.subplots(nrows=2, ncols=3, figsize=(18, 8))

ax0.hist(race_by_age_group_visual[['race_aa_agegrp_1%', 'race_aa_agegrp_2%', 'race_aa_agegrp_3%', 'race_aa_agegrp_4%']], bins=100, histtype='step', color=colors, label=labels, alpha=0.7)
ax0.legend(title='Asian', loc='upper right')

ax1.hist(race_by_age_group_visual[['race_ba_agegrp_1%', 'race_ba_agegrp_2%', 'race_ba_agegrp_3%', 'race_ba_agegrp_4%']], bins=100, histtype='step', color=colors, label=labels, alpha=0.7)
ax1.legend(title='Black or African American', loc='upper right')

ax3.hist(race_by_age_group_visual[['race_wa_agegrp_1%', 'race_wa_agegrp_2%', 'race_wa_agegrp_3%', 'race_wa_agegrp_4%']], bins=100, histtype='step', color=colors, label=labels, alpha=0.7)
ax3.legend(title='White', loc='upper right')

ax4.hist(race_by_age_group_visual[['race_ia_agegrp_1%', 'race_ia_agegrp_2%', 'race_ia_agegrp_3%', 'race_ia_agegrp_4%']], bins=100, histtype='step', color=colors, label=labels, alpha=0.7)
ax4.legend(title='American Indian and Alaska Native', loc='upper right')

ax5.hist(race_by_age_group_visual[['race_na_agegrp_1%', 'race_na_agegrp_2%', 'race_na_agegrp_3%', 'race_na_agegrp_4%']], bins=100, histtype='step', color=colors, label=labels, alpha=0.7)
ax5.legend(title='Native Hawaiian and Other Pacific Islander', loc='upper right')

ax6.hist(race_by_age_group_visual[['race_tom_agegrp_1%', 'race_tom_agegrp_2%', 'race_tom_agegrp_3%', 'race_tom_agegrp_4%']], bins=100, histtype='step', color=colors, label=labels, alpha=0.7)
ax6.legend(title='Two or More Races', loc='upper right')

fig.text(0.5, 0.04, '% of people of age X from a total population of a race Y', ha='center', va='center', fontsize=16)
fig.text(0.06, 0.5, 'n of counties', ha='center', va='center', rotation='vertical', fontsize=16)


⬆️ The following figure shows the percent of people of a given **race** in a given **age group** to the total number of people of that race in a county. For example, similar proportions of people of all age groups, means that this population has a long life length. On the other hand, if the percentage of older people is smaller, that would mean that people's life length is smaller.


We can see from the figure above that for representatives of **all races have shorter life length than people of white race**. For white race the percentage of  people above 65 years old is bigger, compared to other races. This might mean that people of other races in the majority of cases have  worse living conditions than people of white race.

### Poverty

Median proportion of people in poverty across US counties is around 13%. The maximum poverty proportion reaches a value of almost 44% (of total county population). Majority of US counties have poverty level below 17% ⬇️

In [ ]:
print('Shape of poverty dataset is: ', pov_data.shape)
pov_data.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.ion()
sns.set(context="paper")

pov_barplot = sns.barplot(data=pov_data.nlargest(100, 'pctpovall_2020'), x='stabr', y='pctpovall_2020', palette='muted')
ticks = plt.setp(pov_barplot.get_xticklabels(), rotation=0, size=10)
pov_barplot.set_ylabel('people in poverty, \n% of total county population\n (average)', fontsize=12)
pov_barplot.set_xlabel('', fontsize=12)
title = pov_barplot.set_title('Counties with the highest poverty level by state\n', fontsize=20)
plt.show()

These states include 100 counties with the highest poverty levels, grouped by states. The bar plot shows the average percentage of poverty, and  vertical lines show the 95% CI.

- SD - South Dakota
- MS - Mississippi
- GA - Georgia
- LA - Louisiana
- KY - Kentucky
- CO - Colorado
- AR - Arkansas
- TN - Tennessee
- MO - Missouri
- AZ - Arizona
- NM - New Mexico
- WV - West Virginia
- SC - South Carolina
- AL - Alabama
- NC - North Carolina
- MT - Montana
- TX - Texas
- ND - North Dakota
- AK - Alaska
- VA - Virginia



In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
import pandas as pd
import plotly.express as px

fig = px.choropleth_mapbox(data, geojson=counties, locations='countyfips', color='pctpovall_2020',
                           color_continuous_scale="Viridis",
                           range_color=(3, 20),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'pctpovall_2020':'poverty rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

⬆️ The map shows poverty rates across US counties.
The patterns of poverty rates and lack of health insurance rates partially coincides. This is corresponds with the intuition behind this phenomenon: **people below poverty level tend not to buy health insurance plans**.

### Education and general health

In the majority of US counties around 33% of adults have only a high school diploma, around 31% completed some college or associate's degree, 21% have bachelor's degree or higher, and around 11% of adults do not have a high school diploma.

In [ ]:
print('Shape of education dataset is: ', edu_data.shape)
edu_data.describe()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

fig, ((ax1,ax2)) = plt.subplots(nrows=1, ncols=2, figsize=(12, 5), sharey=False)
fig.suptitle("Education and 'poor general health' rates", fontsize=15)

edu_scatter1 = ax1.scatter(x=data['adults_no_hs_diploma_percent'], y=data['adults_hs_diploma_only_percent'], \
                           c=data['ghlth_crudeprev'], s=7, vmin=5, vmax=30, alpha=0.6, cmap='viridis')
ax1.set_xlabel('adults with no high school diploma, %', fontsize=12)
ax1.set_ylabel('adults with high school diploma only, %', fontsize=10)
ax1.set_facecolor('#f7f7f7')
ax1.set_title('Figure 1')
plt.colorbar(edu_scatter1)

edu_scatter2 = ax2.scatter(x=data['adults_college_diploma_percent'], y=data['adults_bachelors_or_higher_percent'], \
                           c=data['ghlth_crudeprev'], s=7, vmin=5, vmax=30, alpha=0.6, cmap='viridis')
ax2.set_xlabel("adults completing some \n college or associate's degree, %", fontsize=12)
ax2.set_ylabel("adults with \na bachelor's degree or higher, %", fontsize=10)
ax2.set_facecolor('#f7f7f7')
ax2.set_title('Figure 2')
plt.colorbar(edu_scatter2)

fig.tight_layout()

⬆️ Level of education and general health.


*Note, that*
- 🟡 yellow color denotes high rates of people with poor self-reported general health (or not good general health in a county population)
- 🟣 purple color denotes low rates of poor self-reported general health (or good general health in a county population).


From Figure 1, we see that counties with high percent of people **not having high school diploma** characterized by high rates of **'poor general health'**. If a big part of the population in a county has only a high school diploma, it does not affect health rates much. However, if there are big percent of people without high school diplomas AND with only high school diplomas - it negatively affects health rates, in other words, yields high 'poor general health' rates.


From this we can infer, that **level of education** is **positively associated with health rates**. 

From Figure 2 we can see that indeed, the more people in a county have completed some college and have received at least bachelor's degree, the better health rates (purple color means low "poor general health" rates).

*Note*, that it does NOT mean that low levels of education cause poor general health. The poverty can be causing both: low level of education and poor health*.

### Mental health and median household income

In [ ]:
print('Shape of unemployment dataset is: ', data.shape)
data[['countyfips',	'unemployment_rate_2020',	'median_household_income_2020']].describe()

In [ ]:
data[['depression_crudeprev', 'mhlth_crudeprev']].describe()

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
import pandas as pd
import plotly.express as px

fig = px.choropleth_mapbox(data, geojson=counties, locations='countyfips', color='mhlth_crudeprev',
                           color_continuous_scale="Viridis",
                           range_color=(8, 19),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'mhlth_crudeprev':'mental health \nnot good rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
import pandas as pd
import plotly.express as px

fig = px.choropleth_mapbox(data, geojson=counties, locations='countyfips', color='median_household_income_2020',
                           color_continuous_scale="Viridis",
                           range_color=(25000, 100000),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'mhlth_crudeprev':'mental health \nnot good rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

The above two maps shows the median **household income** rates and **poor mental health** rates across US counties.


According to the maps, there is **no strong linear correlation** between these two factors.


However, in some areas with high median household income, such as **District of Columbia, New York,** and **San Francisco**, **mental health** rates are **better**. Similarly, in most of counties located in **Louisiana, Alabama, Georgia, Kentucky** where the median household income is low, **poor mental health** rates are high.

## 3. Algorithm Task
Using the data you created in `Section 1.0` and based on your insights from `Section 2.0`, train a Machine learning algorithm to predict health issues in the county. More specficially, you will need to 

1. Create a target variable that measures the prevalence of health issues in US counties.
2. Create a feature matrix of factors that you believe will predit the target variable.
3. Train a machine learning model to predict the target variable from the feature matrix.
3. Choose a metric to assess the performance of the trained machine learning algorithm.
4. Rank-order the importance of the selected features.

You must be able to justify all your choices. 

### Target variable

In [ ]:
# group PLACES dataset columns by their meanings
risk_behaviors = ['binge_crudeprev', 'csmoking_crudeprev', 'sleep_crudeprev', 'lpa_crudeprev']
prevention = ['access2_crudeprev', 'checkup_crudeprev', 'dental_crudeprev', 'bpmed_crudeprev', 'cholscreen_crudeprev', 'mammouse_crudeprev', 'cervical_crudeprev',\
              'corem_crudeprev', 'corew_crudeprev', 'colon_screen_crudeprev']
outcomes = ['arthritis_crudeprev', 'casthma_crudeprev', 'bphigh_crudeprev', 'cancer_crudeprev', 'highchol_crudeprev', 'kidney_crudeprev', 'copd_crudeprev',\
            'chd_crudeprev', 'diabetes_crudeprev', 'depression_crudeprev', 'obesity_crudeprev', 'stroke_crudeprev', 'teethlost_crudeprev']
health_status = ['ghlth_crudeprev', 'phlth_crudeprev', 'mhlth_crudeprev']

To construct target variables, we use supplementary mortality dataset and health outcomes columns from PLACES.


We use mortality rates as a direction to weight particular health issues for a county.


The more a health issue (health outcome) rate is correlated with mortality rate - the higher weight we assign to this particular health issue.


The target variable is constructed in the following way:


1. Each feature column multiplied by the correlation coefficient between this feature and mortality rate.
3. All feature columns are summed up, resulting into one new feature column.
4. The resulting variables are scaled between 0 and 1 using the min max scaling method.
5. Substract the resulting variable from 1 to change the direction from 0 to 1, where 0 corresponds to poor population health and 1 corresponds to good population health.

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

# plot feature correlation heatmap
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,12))
title = fig.suptitle('Correlation of different health outcomes rates with mortlity rates\n', fontsize=16)

data_mort_outcomes_corr = data[[*outcomes, *['mort_cruderate']]].corr(method='spearman')
corr_map = sns.heatmap(data_mort_outcomes_corr, annot=True, cmap="BuPu", ax=ax)

fig.tight_layout()

⬆️ The following heatmap shows correlation between different health issues rates and mortality rates. The most correlated with mortality rates health issues are *coronary heart disease, stroke, kidney disease, high blood pressure, arthritis, high cholesterol, chronic obstructive pulmonary disease, diabetes*.


We also see that some of the features are highly correlated with each other, such as stroke and kidney disease, diabetes and kidney disease, coronary heart disease and kidney disease, coronary heart disease and  stroke.

In [ ]:
# build a dictionary with correlation coeficients for mortality and each health issue
outcomes_mort_coef_map = dict(data_mort_outcomes_corr.loc[:, 'mort_cruderate'][:-1].items())
# multiply each column by the coefficient and sum up 
res = data[outcomes].mul(outcomes_mort_coef_map, axis=1).sum(axis=1).round(decimals=2)
# create a dataframe with target variable and mortality rates (for visualizing)
target_data = pd.DataFrame()
target_data['target_var'] = res.values
target_data['mort_cruderate'] = data['mort_cruderate']
target_data['countyfips'] = data['countyfips']
# scale the resulting variable between 0 and 1
min_, max_ = min(res), max(res)
target_data['target_var'] = target_data.target_var.apply(lambda x: 1 - (x-min_)/(max_ - min_))

In [ ]:
# visualizing the relatipnships between two variables
fig, ax1 = plt.subplots(figsize=(8, 5))
plt1 = sns.scatterplot(target_data, x='target_var',y='mort_cruderate', ax=ax1)
_ = plt1.set_xlabel('target variable', fontsize=14)
_ = plt1.set_ylabel('mortality rates', fontsize=14)
_ = fig.suptitle('Relationships between target variable and mortality rates', fontsize=16)

From the figure above we can notice that the relationships between target variable and mortality rate are close to **linear**.

### Feature matrix

In [ ]:
# these columns do not yield unique valuable information
## data.select_dtypes(include=['object'])
cols_to_drop = [ 'stateabbr','statedesc','countyname','geolocation','population_x','notes',
 'state code',
 'state',
 'county',
 'year',
 'year code',
 'deaths',
 'population_y',
 'crude rate lower 95% confidence interval',
 'crude rate upper 95% confidence interval',
 'stabr',
 'area_name','place',
  ]

# remove this columns only for visualization purposes
flags = ['f_disabl', 'f_sngpnt', 'f_minrty', 'f_limeng', 'f_noveh', 'f_pov']

age_groups_123 = ['race_aa_agegrp_1%', 'race_aa_agegrp_2%', 'race_aa_agegrp_3%','race_ba_agegrp_1%', 'race_ba_agegrp_2%', 'race_ba_agegrp_3%',\
                  'race_ia_agegrp_1%', 'race_ia_agegrp_2%', 'race_ia_agegrp_3%','race_na_agegrp_1%', 'race_na_agegrp_2%', 'race_na_agegrp_3%',\
                  'race_wa_agegrp_1%', 'race_wa_agegrp_2%', 'race_wa_agegrp_3%','race_tom_agegrp_1%', 'race_tom_agegrp_2%', 'race_tom_agegrp_3%']

race_cols = ['race_wa', 'race_ba', 'race_ia', 'race_aa', 'race_na', 'race_tom']

# remove some coulms which are highly correlated with another columns
columns_high_corr = ['totalpopulation', 'povall_2020', 'ep_pov', 'total hours', '% sun']

In [ ]:
# remove unnecesary columns and columns related to target variable
feature_matrix = data.drop(columns=[*['mort_cruderate'], *outcomes, *health_status, *cols_to_drop])
print('We have {} features.'.format(feature_matrix.shape[1]))
print(feature_matrix.columns.to_list())

#### Feature correlation heatmap

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

# plot feature correlation heatmap
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20,20))
title = fig.suptitle('Features correlation\n', fontsize=16)

features_corr = feature_matrix.drop(columns=[*['countyfips'], *flags, *age_groups_123, *race_cols]).corr(method='spearman')
corr_map = sns.heatmap(features_corr, annot=False, cmap="BuPu", ax=ax)

fig.tight_layout()

In [ ]:
# check dtypes of data columns
feature_matrix.info()

The target variable value ranges between 0.0 and 1.0, where 0.0 is the low risk of bad health outcomes, and 1.0 is a high risk of bad health outcomes in a county.


We can divide it into several classes, depending on the task and objectives, for example, values between 0.00-0.29 define "low risk of high rates of adverse health outcomes in a county", 0.30-0.59 define "moderate risk of high rates of adverse health outcomes in a county", and 0.60-1.00 define "high risk of high rates of adverse health outcomes in a county".


For now, we will use the direct variable value for model building.

### Machine learning model building

 #### **Problem settings**

- Task: Since we choose a continuous target variable, the prediction task is **regression**.


- The model used in this study is **XGBoost regressor**, since it is one of the state-of-the-art models for tabular data problems, it is fast to train, and it is well interpretable.

- Each sample in the training/testing set is a **county**. 

- Majority of features are crude prevalence rates on **county level**, however, some features are of **state-level**.

#### Functions

First, we create some functions to facilitate the model building process.

In [ ]:
# Splitting and preparing the data for being passed to the model
from sklearn.model_selection import train_test_split

def split_data(feature_matrix, var_names=None, test_size=0.2, random_seed=42, verbose=True):
  # we remove categorical variables for now
  categorical = ['rural_urban_cont_code', 'urban_inf_code']
  if var_names==None:
    # save features' names
    var_names = feature_matrix.drop(columns=[*['countyfips'], *categorical]).columns

  # data split
  X_train, X_test, y_train, y_test = train_test_split(feature_matrix[var_names], target_data[['target_var']], \
                                                      test_size=test_size, random_state=random_seed)

  # impute categorical variables with zeros
  X_train.loc[:, X_train.columns.str.contains('^f', regex=True)] = X_train.loc[:, X_train.columns.str.contains('^f', regex=True)].fillna(0)
  X_test.loc[:, X_test.columns.str.contains('^f', regex=True)] = X_test.loc[:, X_test.columns.str.contains('^f', regex=True)].fillna(0)
  # impute continuous variables with median
  X_train = X_train.fillna(X_train.median())
  X_test = X_test.fillna(X_test.median())

  if verbose==True:
    print('number of NaNs in X_train: {}, X_test:  {}'.format(X_train.isna().sum().sum(), X_test.isna().sum().sum()))
    print('Shapes, X_train: {}, X_test: {}, y_train: {}, y_test {}'.format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

  # convert to arrays
  X_train, X_test, y_train, y_test = np.asarray(X_train.values), np.asarray(X_test.values), np.asarray(y_train.values).ravel(), np.asarray(y_test.values).ravel()

  return (X_train, X_test, y_train, y_test), var_names

In [ ]:
from sklearn import ensemble
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# this function wraps up the model creating, model fitting, and printing the results
def fit_xgb(X_train, X_test, y_train, y_test, params=None, metric='mae', random_seed=42):
  if params==None:
    params = {
        "n_estimators": 500,
        "max_depth": 4,
        "min_samples_split": 5,
        "learning_rate": 0.05,
        "loss": "absolute_error",
        "random_state": random_seed
    }

  xgb =  ensemble.GradientBoostingRegressor(**params)

  xgb.fit(X_train, y_train)

  if metric=='mse':
    mse = mean_squared_error(y_test.reshape(-1, 1), xgb.predict(X_test))
    print("The mean squared error (MSE) on test set: {:.4f}".format(mse))
  if metric=='mae':
    mae = mean_absolute_error(y_test.reshape(-1, 1), xgb.predict(X_test))
  print("\nThe mean absolute error (MAE) on test set: {:.4f}".format(mae))
  return xgb

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# this function plots the true test values against predicted values
def plot_results(model):
  pred = model.predict(X_test)
  mae = np.round(mean_absolute_error(y_test, pred), 3)
  r2 = np.round(r2_score(y_test, pred), 3)
  # mse = np.round(mean_squared_error(y_test, pred), 3)

  print('MAE {},  R2 {}'.format(mae, r2))

  plt.figure(figsize=(7,7))
  plt.scatter(y_test, pred, c='#3b5998', s=50)
  p1 = max(max(pred), max(y_test))
  p2 = min(min(pred), min(y_test))
  plt.plot([p1, p2], [p1, p2], 'b-')
  plt.xlabel('True Values', fontsize=15)
  plt.ylabel('Predictions', fontsize=15)
  plt.axis('equal')
  plt.title('MAE {},  R2 {}'.format(mae, r2), fontdict={'size':15})
  # Adding text on the plot.
  plt.show()

In [ ]:
# this function plots loss after each iteration on train and on test sets
def plot_deviance(model, params=None):
  test_score = np.zeros((params["n_estimators"],), dtype=np.float64)
  for i, y_pred in enumerate(model.staged_predict(X_test)):
      test_score[i] = mean_absolute_error(y_test, y_pred)

  fig = plt.figure(figsize=(6, 6))
  plt.subplot(1, 1, 1)
  plt.title("Deviance")
  plt.plot(
      np.arange(params["n_estimators"]) + 1,
      model.train_score_,
      "b-",
      label="Training Set Deviance",
  )
  plt.plot(
      np.arange(params["n_estimators"]) + 1, test_score, "r-", label="Test Set Deviance"
  )
  plt.legend(loc="upper right")
  plt.xlabel("Boosting Iterations")
  plt.ylabel("Deviance")
  fig.tight_layout()
  plt.show()

In [ ]:
from sklearn.inspection import permutation_importance

# this function plots importances usinf the impurity-based and permutation methods
def plot_importances(model, var_names, return_permutation=True):
  feature_importance = model.feature_importances_
  sorted_idx1 = np.argsort(feature_importance)
  pos = np.arange(sorted_idx1.shape[0]) + 0.5

  fig_size = (12, (len(var_names)/100)*20,)
  print('figsize: ', fig_size)
  fig = plt.figure(figsize=fig_size)
  plt.subplot(1, 2, 1)
  plt.barh(pos, feature_importance[sorted_idx1], align="center")
  plt.yticks(pos, np.array(var_names)[sorted_idx1])
  plt.title("Feature Importance (MDI)")

  result = permutation_importance(
      model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
  )
  sorted_idx2 = result.importances_mean.argsort()
  plt.subplot(1, 2, 2)
  plt.boxplot(
      result.importances[sorted_idx2].T,
      vert=False,
      labels=np.array(var_names)[sorted_idx2],
  )
  plt.title("Permutation Importance (test set)")
  fig.tight_layout()
  plt.show()

  if return_permutation:
    return np.array(var_names)[sorted_idx2]
  else:
    return np.array(var_names)[sorted_idx1]

This function is taken from the *yellowbrick* visualization library source code, since the normal import yielded some errors. 

In [ ]:
# yellowbrick.regressor.residuals
# Visualize the residuals between predicted and actual data for regression problems
#
# Author:   Rebecca Bilbro
# Author:   Benjamin Bengfort
# Created:  Fri Jun 03 10:30:36 2016 -0700
#
# Copyright (C) 2016 The scikit-yb developers
# For license information, see LICENSE.txt
#
# ID: residuals.py [7d3f5e6] benjamin@bengfort.com $

"""
Visualize the residuals between predicted and actual data for regression problems
"""

import matplotlib.pyplot as plt

from scipy.stats import probplot

try:
    # Only available in Matplotlib >= 2.0.2
    from mpl_toolkits.axes_grid1 import make_axes_locatable
except ImportError:
    make_axes_locatable = None

from yellowbrick.draw import manual_legend
from yellowbrick.utils.decorators import memoized
from yellowbrick.style.palettes import LINE_COLOR
from yellowbrick.exceptions import YellowbrickValueError
from yellowbrick.regressor.base import RegressionScoreVisualizer

class ResidualsPlot(RegressionScoreVisualizer):
    """
    A residual plot shows the residuals on the vertical axis and the
    independent variable on the horizontal axis.
    If the points are randomly dispersed around the horizontal axis, a linear
    regression model is appropriate for the data; otherwise, a non-linear
    model is more appropriate.
    Parameters
    ----------
    estimator : a Scikit-Learn regressor
        Should be an instance of a regressor, otherwise will raise a
        YellowbrickTypeError exception on instantiation.
        If the estimator is not fitted, it is fit when the visualizer is fitted,
        unless otherwise specified by ``is_fitted``.
    ax : matplotlib Axes, default: None
        The axes to plot the figure on. If None is passed in the current axes
        will be used (or generated if required).
    hist : {True, False, None, 'density', 'frequency'}, default: True
        Draw a histogram showing the distribution of the residuals on the
        right side of the figure. Requires Matplotlib >= 2.0.2.
        If set to 'density', the probability density function will be plotted.
        If set to True or 'frequency' then the frequency will be plotted.
    qqplot : {True, False}, default: False
        Draw a Q-Q plot on the right side of the figure, comparing the quantiles
        of the residuals against quantiles of a standard normal distribution.
        Q-Q plot and histogram of residuals can not be plotted simultaneously,
        either `hist` or `qqplot` has to be set to False.
    train_color : color, default: 'b'
        Residuals for training data are ploted with this color but also
        given an opacity of 0.5 to ensure that the test data residuals
        are more visible. Can be any matplotlib color.
    test_color : color, default: 'g'
        Residuals for test data are plotted with this color. In order to
        create generalizable models, reserved test data residuals are of
        the most analytical interest, so these points are highlighted by
        having full opacity. Can be any matplotlib color.
    line_color : color, default: dark grey
        Defines the color of the zero error line, can be any matplotlib color.
    train_alpha : float, default: 0.75
        Specify a transparency for traininig data, where 1 is completely opaque
        and 0 is completely transparent. This property makes densely clustered
        points more visible.
    test_alpha : float, default: 0.75
        Specify a transparency for test data, where 1 is completely opaque
        and 0 is completely transparent. This property makes densely clustered
        points more visible.
    is_fitted : bool or str, default='auto'
        Specify if the wrapped estimator is already fitted. If False, the estimator
        will be fit when the visualizer is fit, otherwise, the estimator will not be
        modified. If 'auto' (default), a helper method will check if the estimator
        is fitted before fitting it again.
    kwargs : dict
        Keyword arguments that are passed to the base class and may influence
        the visualization as defined in other Visualizers.
    Attributes
    ----------
    train_score_ : float
        The R^2 score that specifies the goodness of fit of the underlying
        regression model to the training data.
    test_score_ : float
        The R^2 score that specifies the goodness of fit of the underlying
        regression model to the test data.
    Examples
    --------
    >>> from yellowbrick.regressor import ResidualsPlot
    >>> from sklearn.linear_model import Ridge
    >>> model = ResidualsPlot(Ridge())
    >>> model.fit(X_train, y_train)
    >>> model.score(X_test, y_test)
    >>> model.show()
    Notes
    -----
    ResidualsPlot is a ScoreVisualizer, meaning that it wraps a model and
    its primary entry point is the ``score()`` method.
    The residuals histogram feature requires matplotlib 2.0.2 or greater.
    """

    def __init__(
        self,
        estimator,
        ax=None,
        hist=True,
        qqplot=False,
        train_color="b",
        test_color="g",
        line_color=LINE_COLOR,
        train_alpha=0.75,
        test_alpha=0.75,
        is_fitted="auto",
        **kwargs
    ):

        # Initialize the visualizer base
        super(ResidualsPlot, self).__init__(
            estimator,
            ax=ax,
            is_fitted=is_fitted,
            **kwargs)

        # TODO: allow more scatter plot arguments for train and test points
        # See #475 (RE: ScatterPlotMixin)
        self.colors = {
            "train_point": train_color,
            "test_point": test_color,
            "line": line_color,
        }

        self.hist = hist
        if self.hist not in {True, "density", "frequency", None, False}:
            raise YellowbrickValueError(
                "'{}' is an invalid argument for hist, use None, True, "
                "False, 'density', or 'frequency'".format(hist)
            )

        self.qqplot = qqplot
        if self.qqplot not in {True, False}:
            raise YellowbrickValueError(
                "'{}' is an invalid argument for qqplot, use True, "
                " or False".format(hist)
            )

        if self.hist in {True, "density", "frequency"} and self.qqplot in {True}:
            raise YellowbrickValueError(
                "Set either hist or qqplot to False, can not plot "
                "both of them simultaneously."
            )

        if self.hist in {True, "density", "frequency"}:
            self.hax  # If hist is True, test the version availability

        if self.qqplot in {True}:
            self.qqax  # If qqplot is True, test the version availability

        # Store labels and colors for the legend ordered by call
        self._labels, self._colors = [], []

        self.alphas = {"train_point": train_alpha, "test_point": test_alpha}

    @memoized
    def hax(self):
        """
        Returns the histogram axes, creating it only on demand.
        """
        if make_axes_locatable is None:
            raise YellowbrickValueError(
                (
                    "residuals histogram requires matplotlib 2.0.2 or greater "
                    "please upgrade matplotlib or set hist=False on the visualizer"
                )
            )

        divider = make_axes_locatable(self.ax)

        hax = divider.append_axes("right", size=1, pad=0.1, sharey=self.ax)
        hax.yaxis.tick_right()
        hax.grid(False, axis="x")

        return hax

    @memoized
    def qqax(self):
        """
        Returns the Q-Q plot axes, creating it only on demand.
        """
        if make_axes_locatable is None:
            raise YellowbrickValueError(
                (
                    "residuals histogram requires matplotlib 2.0.2 or greater "
                    "please upgrade matplotlib or set qqplot=False on the visualizer"
                )
            )

        divider = make_axes_locatable(self.ax)

        qqax = divider.append_axes("right", size=2, pad=0.25, sharey=self.ax)
        qqax.yaxis.tick_right()

        return qqax

    def fit(self, X, y, **kwargs):
        """
        Parameters
        ----------
        X : ndarray or DataFrame of shape n x m
            A matrix of n instances with m features
        y : ndarray or Series of length n
            An array or series of target values
        kwargs: keyword arguments passed to Scikit-Learn API.
        Returns
        -------
        self : ResidualsPlot
            The visualizer instance
        """
        # fit the underlying model to the data
        super(ResidualsPlot, self).fit(X, y, **kwargs)
        self.score(X, y, train=True)
        return self

    def score(self, X, y=None, train=False, **kwargs):
        """
        Generates predicted target values using the Scikit-Learn
        estimator.
        Parameters
        ----------
        X : array-like
            X (also X_test) are the dependent variables of test set to predict
        y : array-like
            y (also y_test) is the independent actual variables to score against
        train : boolean
            If False, `score` assumes that the residual points being plotted
            are from the test data; if True, `score` assumes the residuals
            are the train data.
        Returns
        -------
        score : float
            The score of the underlying estimator, usually the R-squared score
            for regression estimators.
        """
        # Do not call super in order to differentiate train and test scores.
        score = self.estimator.score(X, y, **kwargs)
        if train:
            self.train_score_ = score
        else:
            self.test_score_ = score

        y_pred = self.predict(X)
        residuals = y_pred - y
        self.draw(y_pred, residuals, train=train)

        return score

    def draw(self, y_pred, residuals, train=False, **kwargs):
        """
        Draw the residuals against the predicted value for the specified split.
        It is best to draw the training split first, then the test split so
        that the test split (usually smaller) is above the training split;
        particularly if the histogram is turned on.
        Parameters
        ----------
        y_pred : ndarray or Series of length n
            An array or series of predicted target values
        residuals : ndarray or Series of length n
            An array or series of the difference between the predicted and the
            target values
        train : boolean, default: False
            If False, `draw` assumes that the residual points being plotted
            are from the test data; if True, `draw` assumes the residuals
            are the train data.
        Returns
        -------
        ax : matplotlib Axes
            The axis with the plotted figure
        """

        if train:
            color = self.colors["train_point"]
            label = "Train $R^2 = {:0.3f}$".format(self.train_score_)
            alpha = self.alphas["train_point"]
        else:
            color = self.colors["test_point"]
            label = "Test $R^2 = {:0.3f}$".format(self.test_score_)
            alpha = self.alphas["test_point"]

        # Update the legend information
        self._labels.append(label)
        self._colors.append(color)

        # Draw the residuals scatter plot
        self.ax.scatter(y_pred, residuals, c=color, alpha=alpha, label=label)

        # Add residuals histogram
        if self.hist in {True, "frequency"}:
            self.hax.hist(residuals, bins=50, orientation="horizontal", color=color)
        elif self.hist == "density":
            self.hax.hist(
                residuals, bins=50, orientation="horizontal", density=True, color=color
            )

        # Add residuals histogram
        if self.qqplot in {True}:
            osm, osr = probplot(residuals, dist="norm", fit=False)

            self.qqax.scatter(osm, osr, c=color, alpha=alpha, label=label)

        # Ensure the current axes is always the main residuals axes
        plt.sca(self.ax)
        return self.ax

    def finalize(self, **kwargs):
        """
        Prepares the plot for rendering by adding a title, legend, and axis labels.
        Also draws a line at the zero residuals to show the baseline.
        Parameters
        ----------
        kwargs: generic keyword arguments.
        Notes
        -----
        Generally this method is called from show and not directly by the user.
        """
        # Add the title to the plot
        self.set_title("Residuals for {} Model".format(self.name))

        # Set the legend with full opacity patches using manual legend
        manual_legend(self, self._labels, self._colors, loc="best", frameon=True)

        # Create a full line across the figure at zero error.
        self.ax.axhline(y=0, c=self.colors["line"])

        # Set the axes labels
        self.ax.set_ylabel("Residuals")
        self.ax.set_xlabel("Predicted Value")

        # Finalize the histogram axes
        if self.hist:
            self.hax.axhline(y=0, c=self.colors["line"])
            self.hax.set_xlabel("Distribution")

        # Finalize the histogram axes
        if self.qqplot:
            self.qqax.set_title("Q-Q plot")
            self.qqax.set_xlabel("Theoretical quantiles")
            self.qqax.set_ylabel("Observed quantiles")
            
def residuals_plot(
    estimator,
    X_train,
    y_train,
    X_test=None,
    y_test=None,
    ax=None,
    hist=True,
    qqplot=False,
    train_color="b",
    test_color="g",
    line_color=LINE_COLOR,
    train_alpha=0.75,
    test_alpha=0.75,
    is_fitted="auto",
    show=True,
    **kwargs
):
    """ResidualsPlot quick method:
    A residual plot shows the residuals on the vertical axis and the
    independent variable on the horizontal axis.
    If the points are randomly dispersed around the horizontal axis, a linear
    regression model is appropriate for the data; otherwise, a non-linear
    model is more appropriate.
    Parameters
    ----------
    estimator : a Scikit-Learn regressor
        Should be an instance of a regressor, otherwise will raise a
        YellowbrickTypeError exception on instantiation.
        If the estimator is not fitted, it is fit when the visualizer is fitted,
        unless otherwise specified by ``is_fitted``.
    X_train : ndarray or DataFrame of shape n x m
        A feature array of n instances with m features the model is trained on.
        Used to fit the visualizer and also to score the visualizer if test splits are
        not directly specified.
    y_train : ndarray or Series of length n
        An array or series of target or class values. Used to fit the visualizer and
        also to score the visualizer if test splits are not specified.
    X_test : ndarray or DataFrame of shape n x m, default: None
        An optional feature array of n instances with m features that the model
        is scored on if specified, using X_train as the training data.
    y_test : ndarray or Series of length n, default: None
        An optional array or series of target or class values that serve as actual
        labels for X_test for scoring purposes.
    ax : matplotlib Axes, default: None
        The axes to plot the figure on. If None is passed in the current axes
        will be used (or generated if required).
    hist : {True, False, None, 'density', 'frequency'}, default: True
        Draw a histogram showing the distribution of the residuals on the
        right side of the figure. Requires Matplotlib >= 2.0.2.
        If set to 'density', the probability density function will be plotted.
        If set to True or 'frequency' then the frequency will be plotted.
    qqplot : {True, False}, default: False
        Draw a Q-Q plot on the right side of the figure, comparing the quantiles
        of the residuals against quantiles of a standard normal distribution.
        Q-Q plot and histogram of residuals can not be plotted simultaneously,
        either `hist` or `qqplot` has to be set to False.
    train_color : color, default: 'b'
        Residuals for training data are ploted with this color but also
        given an opacity of 0.5 to ensure that the test data residuals
        are more visible. Can be any matplotlib color.
    test_color : color, default: 'g'
        Residuals for test data are plotted with this color. In order to
        create generalizable models, reserved test data residuals are of
        the most analytical interest, so these points are highlighted by
        having full opacity. Can be any matplotlib color.
    line_color : color, default: dark grey
        Defines the color of the zero error line, can be any matplotlib color.
    train_alpha : float, default: 0.75
        Specify a transparency for traininig data, where 1 is completely opaque
        and 0 is completely transparent. This property makes densely clustered
        points more visible.
    test_alpha : float, default: 0.75
        Specify a transparency for test data, where 1 is completely opaque
        and 0 is completely transparent. This property makes densely clustered
        points more visible.
    is_fitted : bool or str, default='auto'
        Specify if the wrapped estimator is already fitted. If False, the estimator
        will be fit when the visualizer is fit, otherwise, the estimator will not be
        modified. If 'auto' (default), a helper method will check if the estimator
        is fitted before fitting it again.
    show: bool, default: True
        If True, calls ``show()``, which in turn calls ``plt.show()`` however you cannot
        call ``plt.savefig`` from this signature, nor ``clear_figure``. If False, simply
        calls ``finalize()``
    kwargs : dict
        Keyword arguments that are passed to the base class and may influence
        the visualization as defined in other Visualizers.
    Returns
    -------
    viz : ResidualsPlot
        Returns the fitted ResidualsPlot that created the figure.
    """

    # Instantiate the visualizer
    viz = ResidualsPlot(
        estimator=estimator,
        ax=ax,
        hist=hist,
        qqplot=qqplot,
        train_color=train_color,
        test_color=test_color,
        line_color=line_color,
        train_alpha=train_alpha,
        test_alpha=test_alpha,
        is_fitted=is_fitted,
        **kwargs
    )

    # Fit the visualizer
    viz.fit(X_train, y_train)

    # Score the visualizer
    if X_test is not None and y_test is not None:
        viz.score(X_test, y_test)
    elif X_test is not None or y_test is not None:
        raise YellowbrickValueError(
            "both X_test and y_test are required if one is specified"
        )
    else:
        viz.score(X_train, y_train)

    # Draw the final visualization
    if show:
        viz.show()
    else:
        viz.finalize()

    # Return the visualizer
    return 

#### Model fit and evaluation (& feature importance)

We will run each model training and evaluation 5 times with random states 42, 43, 44, 45, 46 to get more trustworthy model performance.

Model with all 80 predictors

In [ ]:
# set the parameters for the model
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.05,
    "loss": "absolute_error",
    "random_state":46
}

# fit the model and see the results
(X_train, X_test, y_train, y_test), var_names = split_data(feature_matrix, var_names=None, test_size=0.2, random_seed=42)
model = fit_xgb(X_train, X_test, y_train, y_test, params=params, metric='mae')

In [ ]:
plot_results(model)

In [ ]:
plot_deviance(model, params=params)

In [ ]:
importances = plot_importances(model, var_names, )

Model with **40** the most important predictors

In [ ]:
# choose the most important features to train a new model
important_features = list(importances[-40:])
important_features

In [ ]:
# do training again with selected features
# set the parameters for the model
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.05,
    "loss": "absolute_error",
    "random_state":46
}

# fit the model and see the results
(X_train, X_test, y_train, y_test), var_names = split_data(feature_matrix, var_names=important_features, test_size=0.2, random_seed=42)
model_selected_features = fit_xgb(X_train, X_test, y_train, y_test, params=params, metric='mae')

In [ ]:
plot_results(model_selected_features)

In [ ]:
plot_deviance(model_selected_features, params=params)

In [ ]:
importances = plot_importances(model_selected_features, var_names, )

Model with **30** the most important predictors

In [ ]:
# choose the most important features to train a new model
important_features = list(importances[-30:])
important_features

In [ ]:
# do training again with selected features
# set the parameters for the model
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.05,
    "loss": "absolute_error",
    "random_state":46
}

# fit the model and see the results
(X_train, X_test, y_train, y_test), var_names = split_data(feature_matrix, var_names=important_features, test_size=0.2, random_seed=42)
model_selected_features = fit_xgb(X_train, X_test, y_train, y_test, params=params, metric='mae')

In [ ]:
plot_results(model_selected_features)

In [ ]:
plot_deviance(model_selected_features, params=params)

In [ ]:
importances = plot_importances(model_selected_features, var_names, )

#### Evaluation metrics

In the model training we use **Mean Absolute Error (MAE)**, because of two main factors:
- since it is a regression problem, and MAE directly measures errors between predicted and true values and shows the mean of it;
- because target variables values range between 0.0 and 1.0, therefore utilizing other error metrics, such as Mean Squared Error (MSE) will yield extremely small error values.


In addition, we use the **R-squared** metric which measures the proportion of variance in the target variable that can be explained by the independent variables (predictors).

---

In the real world scenario one useful approach would be to define some classes to explain the target variable. With that in mind, we will convert the continuous target variable values into **three classes**:
- **low risk** of high rates of adverse health outcomes in a county, denoted by 0
- **moderate risk** of high rates of adverse health outcomes in a county, denoted by 1
- **high risk** of high rates of adverse health outcomes in a county, denoted by 2

In [ ]:
# get model predictions
y_pred = model_selected_features.predict(X_test)
y_true = y_test

# convert direct values into three classes
y_pred_cls = [0 if x<0.3 else 1 if x<0.6 else 2 for x in y_pred]
y_true_cls = [0 if x<0.3 else 1 if x<0.6 else 2 for x in y_true]

print('number of class 0 samples:', y_true_cls.count(0))
print('number of class 1 samples:', y_true_cls.count(1))
print('number of class 2 samples:', y_true_cls.count(2))
print('total: ', len(y_pred_cls))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true_cls, y_pred_cls))

In [ ]:
from sklearn.metrics import confusion_matrix

# Creating  a confusion matrix,which compares the y_test and y_pred
cm = confusion_matrix(y_true_cls, y_pred_cls, labels=[0,1,2])
# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['low risk','moderate risk','high risk'], 
                     columns = ['low risk','moderate risk','high risk'])

#Plotting the confusion matrix
plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True, cmap='BuPu', vmin=0, vmax=100, fmt='g')
plt.title('Confusion Matrix', fontsize=14)
plt.ylabel('Actal Values', fontsize=12)
plt.xlabel('Predicted Values', fontsize=12)
plt.show()

The model was able to identify **majority of samples correctly**, with the highest F1 scores for moderate risk samples, since it is the most numerous class.

The model overpredicted 11 low risk samples as at moderate risk. 8 samples belonging at risk class were misclassified as at low or high risk. Finally, 8 samples, belonging to high risk class were classified as at moderate risk. 

No samples at low risk class were classified as at  high risk, and **no samples  at high risk  were classified as at low risk**.



---

#### Evaluation summary

In [ ]:
# performace for each of 5 training rounds
performance_all = {'mae':[0.021, 0.02, 0.021, 0.021, 0.021], 'r2':[0.908, 0.924, 0.911, 0.907, 0.912]}
performance_40 = {'mae':[0.02, 0.02, 0.02, 0.021, 0.021], 'r2':[0.915, 0.917, 0.919, 0.912, 0.916]}
performance_30 = {'mae':[0.02, 0.02, 0.02, 0.021, 0.02], 'r2':[0.924, 0.923, 0.93, 0.914, 0.925]}

# get the mean and std for each model
mae_all_mean = np.round(np.mean(performance_all['mae']), 3)
mae_all_std = np.round(np.std(performance_all['mae']), 4)
mae_40_mean = np.round(np.mean(performance_40['mae']),3)
mae_40_std = np.round(np.std(performance_40['mae']),4)
mae_30_mean = np.round(np.mean(performance_30['mae']),3)
mae_30_std = np.round(np.std(performance_30['mae']),4)

r2_all_mean = np.round(np.mean(performance_all['r2']), 3)
r2_all_std = np.round(np.std(performance_all['r2']), 4)
r2_40_mean = np.round(np.mean(performance_40['r2']),3)
r2_40_std = np.round(np.std(performance_40['r2']),4)
r2_30_mean = np.round(np.mean(performance_30['r2']),3)
r2_30_std = np.round(np.std(performance_30['r2']),4)

print('All features model: MAE {}({}), R2 {}({})'.format(mae_all_mean, mae_all_std, r2_all_mean, r2_all_std))
print('40 features model: MAE {}({}), R2 {}({})'.format(mae_40_mean, mae_40_std, r2_40_mean, r2_40_std))
print('30 features model: MAE {}({}), R2 {}({})'.format(mae_30_mean, mae_30_std, r2_30_mean, r2_30_std))

After training each of the three models with different set of predictors, we got the following results:

- all 80 predictors: **MAE 0.021 (std 0.0004),  R2 0.912 (std 0.0061)**
- 40 the most important predictors: **MAE 0.020 (std 0.0005),  R2 0.916 (std 0.0023)**
- 30 the most important predictors: **MAE 0.020 (std 0.0004),  R2 0.923 (std 0.0052)**

The best results yielded the model trained on 30 the most important predictor variables.

The most important features are health risk behavior rates such as "*currently smoking", "low physical activity", "sleep less than 7 hours", "binge drinking"*, as well as preventive measures rates such as "*taking blood pressure medicines", "cholesterol screening", "routine checkup visits", "cervical cancer screening", "dentist visits*".


Other important predictors are related to **social vulnerability status**, such as percent of people with disabilities, household income, unemployment rate, poverty, limited english, and education level.


In addition, **demographic information** such as percent of people of asian race, elderly people of two or more races, percent of young people of black or african american race, elderly and young american indian and alaska native people, as well as percent of native hawaiian people in the county.

One interesting observation is that model put high importance into percentage of **white race people in elderly** and people **between 20 and 39 years old**. This may be the result of the prevalence of white race population across majority of US counties. Most probably, the model  simply took into consideration this feature because it yields the most information about percent of elderly people in a county. It would be informative to see which predictor the model would choose if we had percentage of each age group regardless of race in the feature set.

Another observation regarding feature importance, is that the model put some importance into features related to average annual sunshine by state, such as
- **% of sun**: the percentage of time between sunrise and sunset that sunshine reaches the ground;
- **Total Hours** is the average number of sunny hours a place normally has in a year;
- **Clear Days** is the average number of days annually when cloud covers at most 30 percent of the sky during daylight hours.


These three variables are correlated, meaning that keeping only one of them would probably yield similar results. In addition, they are state-level variables, meaning that they might bring geographical information to the model, and it would be informative to substitute them with one 'state' variable and to see how it will affect the model's performance.

Key to race coding map: 
- ba - black of african alone
- ia - american indian and alaska native alone 
- aa - asian alone
- na - native hawaiian and other pacific islander alone
- wa - white alone
- tom - two or more races 

Key to age group map:
- age group 0: total
- age group 1: 0-19 
- age group 2: 20-39
- age group 3: 40-64
- age group 0: 65+



---


*Maslenkova Svetlana, March 2023*